<a href="https://colab.research.google.com/github/Phanindra-pvs/github-slideshow/blob/master/FAKE_NEWS_PREDICTION_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





Importing the Dependencies

In [70]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [71]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [72]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [73]:
# loading the dataset to a pandas DataFrame
news_dataset=pd.read_csv('train.csv', engine='python', encoding='utf-8', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 5098: unexpected end of data


In [74]:
news_dataset.shape

(5096, 5)

In [75]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [76]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id          0
title     145
author    488
text       11
label       0
dtype: int64

In [77]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [78]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [79]:
print(news_dataset['content'])

0       Darrell Lucus House Dem Aide: We Didn’t Even S...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
5091    Pam Key Sanders: My Republican Colleagues Are ...
5092    ken williams Re: 55 Reasons Why California Is ...
5093    Adalia Woodbury Supreme Court Decides to Weigh...
5094    Donna Rachel Edmunds EU Adds Cost of Migrants ...
5095    The Sleuth Journal BREAKING: WikiLeaks Just Re...
Name: content, Length: 5096, dtype: object


In [80]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [81]:
print(X)
print(Y)

        id                                              title  \
0        0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
5091  5091  Sanders: My Republican Colleagues Are ’Cowardl...   
5092  5092  Re: 55 Reasons Why California Is The Worst Sta...   
5093  5093  Supreme Court Decides to Weigh in On Transgend...   
5094  5094  EU Adds Cost of Migrants in Turkey to Brexit Bill   
5095  5095  BREAKING: WikiLeaks Just Released Full ISIS Do...   

                    author                                               text  \
0            Darrell Lucus  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          Daniel J. Flynn  Ever get the feeling your lif

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [82]:
port_stem = PorterStemmer()

In [83]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [84]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [85]:
print(news_dataset['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
5091    pam key sander republican colleagu cowardli br...
5092    ken william reason california worst state america
5093    adalia woodburi suprem court decid weigh trans...
5094    donna rachel edmund eu add cost migrant turkey...
5095    sleuth journal break wikileak releas full isi ...
Name: content, Length: 5096, dtype: object


In [86]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [87]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'adalia woodburi suprem court decid weigh transgend right'
 'donna rachel edmund eu add cost migrant turkey brexit bill'
 'sleuth journal break wikileak releas full isi donor list name']


In [88]:
print(Y)

[1 0 1 ... 1 0 1]


In [89]:
Y.shape

(5096,)

In [90]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [91]:
print(X)

  (0, 8322)	0.2823173973093671
  (0, 7123)	0.2553967819739332
  (0, 4781)	0.34472597814645645
  (0, 4634)	0.30665382246316647
  (0, 4147)	0.24636701553622983
  (0, 3782)	0.22415236429258323
  (0, 2670)	0.23813730890634857
  (0, 2055)	0.26680641054928267
  (0, 1954)	0.34472597814645645
  (0, 1605)	0.2472825806940064
  (0, 1351)	0.38457339006034025
  (0, 160)	0.27473516276910437
  (1, 8845)	0.2904213886369504
  (1, 3685)	0.18840195895012107
  (1, 2984)	0.7185452513552217
  (1, 1937)	0.27111842951556764
  (1, 1518)	0.19001987625208472
  (1, 1219)	0.38464312208498525
  (1, 1035)	0.1527965778538923
  (1, 822)	0.2879981924870587
  (2, 8282)	0.4193156540507848
  (2, 5144)	0.4916842395823964
  (2, 3235)	0.34542856502223757
  (2, 2915)	0.38506512579148994
  (2, 1693)	0.4597380060548578
  :	:
  (5093, 7839)	0.3132808324482823
  (5093, 6755)	0.26124832800576103
  (5093, 2010)	0.3739894723996527
  (5093, 1784)	0.29063933117386676
  (5093, 71)	0.41573647116188284
  (5094, 8309)	0.29411803212105525


Splitting the dataset to training & test data

In [92]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [93]:
model = LogisticRegression()

In [94]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [95]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [96]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9752208047105005


In [97]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [98]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9441176470588235


Making a Predictive System

In [99]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [100]:
print(Y_test[3])

0
